# STEPS TO USE THIS NOTEBOOK

## 1. GET GSCDCA-LPDAP-ADMPIM-DEM-BDT FROM [HERE](https://portal.azure.com/?Microsoft_Azure_PIMCommon=true&feature.msaljs=true#view/Microsoft_Azure_PIMCommon/ActivationMenuBlade/~/aadgroup/provider/aadgroup) (THIS RESETS AT DAILY BASIC, NEED TO REQUEST IT AGAIN AFTER 8 HOURS)

## 2.START THE CLUSTER WITH YOUR NAME OR CHANGE THE CLUSTER OWNER FROM THE COMPUTE MENU
### STEPS TO CHANGE CLUSTER OWNER
### 2.1 ![Screenshot 2025-06-06 152220.png](./Screenshot 2025-06-06 152220.png "Screenshot 2025-06-06 152220.png")
### 2.2 ![Screenshot 2025-06-06 152719.png](./Screenshot 2025-06-06 152719.png "Screenshot 2025-06-06 152719.png")
### 2.3 ![Screenshot 2025-06-06 152829.png](./Screenshot 2025-06-06 152829.png "Screenshot 2025-06-06 152829.png")
### 2.4 BACK TO THE NOTEBOOK AND NEXT TO RUN ALL IN THE RIGHT UPPER CORNER PRESS THE CLUSTER MAIN
## THOSE STEPS NEED TO BE AT LEAST CHECKED BEFORE STARTING THE NOTEBOOK, IF NOT THE OWNER OF CLUSTER IT WILL CREATE ERRORS

## 3.ON THE TOP OF THE NOTEBOOK YOU CAN SEE ALL PARAMETERS. 
3.1 USER is the name of the folder created on blob storage: for example :

3.2 BASE__CHOISE is the subfolder of your _test folder: for example :

3.3 FILE_TYPE IS THE FILE TYPE YOU WANT PROCESSED, IF YOU HAVE ONLY .TXT FILES INSIDE FILTER PICK TXT

3.4 WANT_CLEAN_CODE IS A FLAG THAT, IF YOU PICK TRUE, IT WILL PROVIDE YOU 2 OUTPUTS, 1 WITH EXPLINATIONS AND THE OTHER ONE WITH JUST CODE

3.5 PROVIDED_CONTEXT THIS IS WHERE YOU ADD YOUR PROMPT

3.6 FILES THIS IS WHERE YOU SELECT WHICH FILES FROM THE FOLDER YOU WANT PROCESSED, IF YOU WANT ALL FILES SELECT "ALL FILES"

3.7 MODEL_CHOICE THIS IS THE GPT MODEL, O1 PROVIDES BEST ANSWERS

## PLEASE SELECT/UPDATE PARAMETERS IN ORDER FROM LEFT TO RIGHT FOR CORRECT VARIABLE APPENDING

## 4.AFTER COMPLETING THE PARAMETERES PRESS RUN ALL FROM UPPER RIGHT CORNER. THE OUTPUT WILL BE AVALIBLE IN THE LAST CELL AND ALSO ON BLOB STORAGE UNDER YOUR FOLDER, EX:

# AFTER YOU RUN THE WHOLE NOTEBOOK ONCE IT IS NOT NECECSARY TO RUN ALL AGAIN. IF YOU CHANGE THE INPUT PARAMETERS THEY WILL BE DINAMICALLY CHANGED. AFTER RUN ALL YOU CAN RUN ONLY THE LAST CELL, THERE IS A BLUE BUTTON ON THE LEFT UPPER CORNER OF THE CELL.

# IF YOU RECIEVE ANY ERROR, PLEASE REFRESH YOUR WEB BROWSER PAGE, IF ERROR PERSIST LOG OUT AND IN DATABRICKS. IF YOU CANNOT START YOUR CLUSTER VERIFY AGAIN IF YOU HAVE ADMIN ACCESS FROM THE GSCDCA-LPDAP-ADMPIM-DEM-BDT ROLE.

In [0]:
%pip install openai
dbutils.fs.rm("dbfs:/tmp/", True)
%restart_python

In [0]:
from openai import AzureOpenAI
from datetime import datetime
import time
import shutil
import re
from collections import OrderedDict
import sys
import time
import os
from pathlib import Path
from pyspark.sql import Row

client = AzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint="PLACEHOLDER",
    api_key="PLACEHOLDER",
)

client_v2 = AzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint="PLACEHOLDER",
    api_key="PLACEHOLDER",
)

In [0]:
import os
   

# Define the storage account and container name
sa = "stlpdel01dev"
container_name = "codeconverter" 
# Set Azure authentication configuration
spark.conf.set("fs.azure.account.auth.type", "CustomAccessToken")
spark.conf.set("fs.azure.account.custom.token.provider.class", spark.conf.get("spark.databricks.passthrough.adls.gen2.tokenProviderClassName"))

# Set the base URL for the container and directories
base_url = f"abfss://{container_name}@{sa}.dfs.core.windows.net/"

# Print the base URL for verification
# print(base_url)

# #asta trebuie pornit primul doar o singura data
import certifi

scope_name = "OpenAI-scope"
secret_name = "OpenAI-certificate"

secret_value = dbutils.secrets.get(scope=scope_name, key=secret_name)

print("---- Retrieve CA ----")
ca_cert = certifi.where()
print("---- Appending new SSL to CA ----")

with open(ca_cert, "a") as custom_certificate:
    cert_content = custom_certificate.write("\n# Custom appended certificate \n")
    cert_content = custom_certificate.write(secret_value)
    cert_content = custom_certificate.write("\n")

print(f"---- Successfully appended to: {ca_cert} ----")


In [0]:
# Create a dropdown widget for selecting model type (o1 or o1 mini)
dbutils.widgets.dropdown("model_choice", "o1", ["o1", "o1-mini", "o1-preview","gpt-4o"], "Choose model type")
# Retrieve the selected model type
selected_model = dbutils.widgets.get("model_choice")
print(f"Selected model: {selected_model}")

In [0]:
def callmodel4o(deployment_name, prompt):

  start_phrase = prompt
  response = client_v2.chat.completions.create(
      model=deployment_name,  # This refers to the deployed model name (gpt-4o)
      messages=[
          {"role": "system", "content": "You are a code converting specialist between SAP Code and Databricks code."},
          {"role": "user", "content": start_phrase}
      ],
      max_tokens=10000,  # Set the maximum length of the response
      temperature=0.2,  # Lower temperature for more deterministic responses
      top_p=0.8,        # More standard responses
      frequency_penalty=0.1, # Allow code repetition
      presence_penalty=0.0 # Less varied content
  )

  #print(response)
  return response.choices[0].message.content

In [0]:
def callmodelo1(deployment_name, prompt):

  start_phrase = prompt
  response = client_v2.chat.completions.create(
      model=deployment_name,  # This refers to the deployed model name (gpt-4o)
      messages=[
        {
            "role": "user",
            "content": start_phrase
        }
    ]
  )

  #print(response)
  return response.choices[0].message.content


In [0]:
def call_model_o1(deployment_name, prompt, context):
    # Define the start phrase as the user input (XML configuration and prompt)
    start_phrase = prompt  # The prompt you want to send to the model

    # Define the system message that provides the context
    system_message = {
        "role": "system",
        "content": context
        }

    # Make the API call to get the response from the model
    response = client_v2.chat.completions.create(
        model=deployment_name,  # The name of your deployed model (e.g., gpt-4-deployment)
        messages=[
            system_message,  # Add the system message to set the context
            {
                "role": "user",  # User is sending the message
                "content": start_phrase  # The actual prompt content (XML configuration)
            }
        ]
    )
    
    return response.choices[0].message.content  # Return the model's response

In [0]:
def call_model(selected_model, prompt, context):
    if selected_model == "o1":
        return call_model_o1(deployment_name="o1", prompt=prompt, context=context)  # Call the o1 model function
    elif selected_model == "o1-preview":
        return callmodelo1(deployment_name="o1-preview", prompt=prompt)  # Call the o1-preview model function
    elif selected_model == "o1-mini":
        return callmodelo1(deployment_name="o1-mini", prompt=prompt)  # Call the o1-mini model function
    elif selected_model == 'gpt-4o':
        return callmodel4o(deployment_name="gpt-4o", prompt=prompt)
    else:
        raise ValueError("Invalid model choice")

In [0]:
cleaning_context = """You are an expert in converting SAP logic, represented in XML configuration, into Databricks SQL.
                   Your task is to parse the logic in the XML and generate the corresponding Databricks SQL queries."""

# CHANGE THE VARIABLE BELOW TO CHANGE THE PROMPT

In [0]:
instruction = """Correct the SQL code below and optimize it with the best practices of Databricks. Check that the if syntax is correct and there is no character missing, remove the formatting and standardize the syntax.Use lowercase for statements and use new line for every column name or statement.Be sure to not change filters. Put the select between () and use ; at the end and a new line. Be sure to replace syntax to syntax compatible with databricks: for example: in("vz_cd_pa",'70','71') -> vz_cd_pa in ('70','71'). Be sure that all the paranthesys opened are closed and no extra are added. Be sure to not change the order of the columns. Output only the code reviewed and corrected.
 
SQL code:\n"""

In [0]:
def get_parameters(files):
    subdir = "queries/"
    SQL_FILE = subdir + files
    GPT_FILE = "GPT_OutPut/" + SQL_FILE.split("/")[-1].split(".")[0]
    with open(SQL_FILE, "r", encoding="utf-8") as files:
        my_sql = files.read()
    prompt = my_sql
    RowList = [Row(col) for col in prompt.split(';') if col.replace('\n', '').replace(" ", "") != '']; 
    RowList = spark.createDataFrame(RowList).withColumnRenamed("_1", "table_statement")
    return GPT_FILE,prompt,RowList


In [0]:
from concurrent.futures import ThreadPoolExecutor, as_completed  
from pyspark.sql.functions import lit

def chat_gpt(input):
    statements, GPT_FILE = input["statements"], input["GPT_FILE"]
    prompt_single_table = statements.replace('\n','')
    table_name = prompt_single_table.split("as (")[0].replace('create or replace temporary view ', '').replace('  ', '')
    table_path = GPT_FILE
    prompt = prompt_single_table
    if (word in prompt_single_table.lower() for word in ('if','case','join','rank','union')):
        try:
            model_output = call_model(selected_model, prompt=prompt, context=instruction)
            with open(f"{GPT_FILE}.sql", "a", newline="") as c:
                c.write(f"\n\n" + model_output)
        except:
            model_output = statements
            with open(f"{GPT_FILE}.sql", "a", newline="") as c:
                c.write(f"\n\n NOT_MODIFIED \n\n" + model_output)
            
    else:
        model_output = statements
        with open(f"{GPT_FILE}.sql", "a", newline="") as c:
            c.write(f"\n\n ONLY_SELECT \n\n" + model_output)

files = os.listdir("queries/")
for file in files:
    GPT_FILE,prompt,RowList = get_parameters(file)
    input = [{"statements": statement, "GPT_FILE": GPT_FILE} for statement in RowList.select("table_statement").rdd.flatMap(lambda x: x).collect()]
    progress = 0
    max_progress = len(input)
    with ThreadPoolExecutor() as executor:
        future_to_file = executor.map(chat_gpt, input)